In [2]:
from time import sleep

from beamngpy import BeamNGpy, Scenario, Vehicle, set_up_simple_logging
from beamngpy.sensors import Mesh, Damage, Camera

In [3]:
# bng = BeamNGpy("localhost", 25252, home='/home/robotixx/Downloads/BeamNG.tech.v0.36.4.0', debug=True)
# bng.open(launch=True)
bng = BeamNGpy("localhost", 25252)
bng.open(launch=True)

In [33]:
vehicle = Vehicle("ego_vehicle", model="rockbouncer", license="DHILO", color="Red")
vehicle.attach_sensor('damage', Damage())
scenario = Scenario("industrial", "broken_mesh_test", description="Mesh Analysis")
scenario.add_vehicle(
        vehicle, pos=(408.36, -336.56, 35.54), rot_quat=(0,0, 0.99, -0.17)
    )
scenario.make(bng)
bng.settings.set_deterministic(60)
bng.scenario.load(scenario)
bng.scenario.start()
bng.step(60)


In [ ]:
vehicle.send_damage([
        ('front_left_suspension', 1.0, (0,0,0)),
    ])

In [ ]:
vehicle.queue_lua_command('beamstate.breakGroup("front_left_suspension")')

In [ ]:
vehicle.queue_lua_command('obj:setTireBurst(2, true)')

In [ ]:
vehicle.queue_lua_command('beamstate.breakGroup("engine")')

In [30]:
vehicle.get_part_config()

{'partsTree': {'partPath': '/rockbouncer',
  'children': {'licenseplate_design_2_1': {'decisionMethod': 'user-empty',
    'suitablePartNames': ['license_plate_nz_1',
     'license_plate_jungle_rock_island_2_1',
     'license_plate_germany_2_1',
     'license_plate_wca_2_1',
     'license_plate_eca_2_1',
     'license_plate_italy_old_2_1',
     'license_plate_gridmap_2_1',
     'license_plate_italy_2_1',
     'license_plate_Texas_2_1',
     'license_plate_Utah_2_1'],
    'chosenPartName': '',
    'path': '/licenseplate_design_2_1/',
    'unsuitablePartNames': [],
    'id': 'licenseplate_design_2_1'},
   'soundscape_horn': {'decisionMethod': 'default',
    'partPath': '/soundscape_horn/soundscape_horn_9',
    'suitablePartNames': ['soundscape_horn_103',
     'soundscape_horn_102',
     'soundscape_horn_127',
     'soundscape_horn_111',
     'soundscape_horn_110',
     'soundscape_horn_9',
     'soundscape_horn_11',
     'soundscape_horn_128',
     'soundscape_horn_118',
     'soundscape_

In [ ]:
vehicle.poll_sensors

In [ ]:
for k, v in pairs(beamstate.breakGroup) do print(k) end

In [ ]:
vehicles = bng.get_available_vehicles()
vehicles

In [ ]:
vehicle.queue_lua_command('beamstate:breakGroup("wheel_FL")')

In [ ]:
vehicle.get_center_of_gravity()

In [ ]:
vehicle.recover()

In [ ]:
vehicle.queue_lua_command('obj:breakAllBreakGroups()')

In [ ]:
print(vehicle.is_connected)

In [ ]:
bng.step(80)

In [ ]:
vehicle.queue_lua_command('print("Hello from Lua!")')

In [61]:
camera = Camera(
    "front_cam",
    bng,
    vehicle,
    requested_update_time = 0.1,
    is_using_shared_memory=True,
    pos=(-0.3, 1, 2),
    dir=(0, -1, 0),
    field_of_view_y = 90,
    near_far_planes=(0.1, 1000),
    resolution=(1024, 1024),
    is_streaming=True,
    is_render_annotations=True,
    is_render_instance=True,
    is_render_depth=True,
)

In [ ]:
for i in range(41):
    bng.control.step(10)

    raw_readings = camera.stream_raw()

    if i % 10 == 0:
        print("Showing images from the stream...")

        images = camera.stream()

        if camera.is_render_colours:
            images["colour"].show()
        if camera.is_render_annotations:
            images["annotation"].show()
        if camera.is_render_depth:
            images["depth"].show()

In [72]:
vehicle.queue_lua_command('energyStorage.getStorage("mainTank"):setRemainingRatio(0.1)')

In [22]:
vehicle.queue_lua_command('damageTracker.setDamage("engine", "radiatorLeak", true)')

In [6]:
# vehicle.queue_lua_command('for k,v in pairs(damageTracker) do print(k) end')
vehicle.queue_lua_command('for group, items in pairs(damageData) do for name, val in pairs(items) do print(group, name, val) end end')


In [13]:
damage_pairs = [
    ('engine', 'radiatorLeak'),
    ('engine', 'oilLeak'),
    ('engine', 'brokenStarter'),
    ('engine', 'seized'),
    ('engine', 'engineFire'),
    ('fuel', 'leak'),
    ('brakes', 'brakeFluidLeak'),
    ('transmission', 'clutchFailure'),
    ('drivetrain', 'rearDifferential'),
]

for group, name in damage_pairs:
    vehicle.queue_lua_command(f'print({group})')
    vehicle.queue_lua_command(f'damageTracker.setDamage("{group}", "{name}", true)')
    bng.step(120)

In [ ]:
vehicle.queue_lua_command('groups.process()')

In [ ]:
vehicle.queue_lua_command('meshes.processFlexbodies()')

In [32]:
vehicle.queue_lua_command('damageTracker.setDamage("engine", "headGasketDamaged", true)')

In [31]:
vehicle.get_part_config()

{'partsTree': {'partPath': '/rockbouncer',
  'children': {'licenseplate_design_2_1': {'decisionMethod': 'user-empty',
    'suitablePartNames': ['license_plate_nz_1',
     'license_plate_jungle_rock_island_2_1',
     'license_plate_germany_2_1',
     'license_plate_wca_2_1',
     'license_plate_eca_2_1',
     'license_plate_italy_old_2_1',
     'license_plate_gridmap_2_1',
     'license_plate_italy_2_1',
     'license_plate_Texas_2_1',
     'license_plate_Utah_2_1'],
    'chosenPartName': '',
    'path': '/licenseplate_design_2_1/',
    'unsuitablePartNames': [],
    'id': 'licenseplate_design_2_1'},
   'soundscape_horn': {'decisionMethod': 'default',
    'partPath': '/soundscape_horn/soundscape_horn_9',
    'suitablePartNames': ['soundscape_horn_103',
     'soundscape_horn_102',
     'soundscape_horn_127',
     'soundscape_horn_111',
     'soundscape_horn_110',
     'soundscape_horn_9',
     'soundscape_horn_11',
     'soundscape_horn_128',
     'soundscape_horn_118',
     'soundscape_